<a href="https://colab.research.google.com/github/Nurguyan/Ice-Hockey-Match-Outcome-Predictor/blob/main/Ice-Hockey-Match-Outcome-Predictor.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [10]:
import numpy as np
import pandas as pd
import tensorflow as tf

url = 'https://raw.githubusercontent.com/Nurguyan/Ice-Hockey-Match-Outcome-Predictor/main/data/data.csv'

df = pd.read_csv(url)
print(df[:2])

         Date Location Team   GF   GA  ...    Sh%    Sv%    PP%    PK%  Class
0  07/09/2018     Home  LON  247  161  ...  36.85  29.08  26.17  82.06   Loss
1  07/09/2018     Away  TER  223  144  ...  40.69  32.31  32.44  84.10    Win

[2 rows x 12 columns]


In [11]:
from sklearn import preprocessing

#we don't need a date since it should not affect on matches' outcome
df = df.drop(['Date'], axis=1)

#convert categorical data into integers
df = pd.get_dummies(df, columns=['Team'])
df['Location'] = (df['Location'] == 'Home').astype(int)
df['Class'] = (df['Class'] == 'Win').astype(int)

print(df[:5])
df.info()

   Location   GF   GA   GF%  ...  Team_STJ  Team_TER  Team_VAL  Team_WIS
0         1  247  161  5.15  ...         0         0         0         0
1         0  223  144  4.65  ...         0         1         0         0
2         1  202  211  4.21  ...         0         0         0         0
3         0  200  207  4.17  ...         0         0         0         0
4         0  207  151  4.31  ...         1         0         0         0

[5 rows x 21 columns]
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1008 entries, 0 to 1007
Data columns (total 21 columns):
 #   Column    Non-Null Count  Dtype  
---  ------    --------------  -----  
 0   Location  1008 non-null   int64  
 1   GF        1008 non-null   int64  
 2   GA        1008 non-null   int64  
 3   GF%       1008 non-null   float64
 4   GA%       1008 non-null   float64
 5   Sh%       1008 non-null   float64
 6   Sv%       1008 non-null   float64
 7   PP%       1008 non-null   float64
 8   PK%       1008 non-null   float64
 9 

In [3]:
#normalize features
x = df.values
scaler = preprocessing.MinMaxScaler()
x_scaled = scaler.fit_transform(x)
df = pd.DataFrame(x_scaled)
print(df[:5])

    0         1         2         3         4   ...   16   17   18   19   20
0  1.0  0.728682  0.268293  0.728625  0.266862  ...  0.0  0.0  0.0  0.0  0.0
1  0.0  0.542636  0.164634  0.542751  0.164223  ...  0.0  0.0  1.0  0.0  0.0
2  1.0  0.379845  0.573171  0.379182  0.574780  ...  0.0  0.0  0.0  0.0  0.0
3  0.0  0.364341  0.548780  0.364312  0.548387  ...  0.0  0.0  0.0  0.0  0.0
4  0.0  0.418605  0.207317  0.416357  0.208211  ...  0.0  1.0  0.0  0.0  0.0

[5 rows x 21 columns]


In [4]:
#split data to features and target
target = df.pop(9) #pop "Class" as target
dataset = tf.data.Dataset.from_tensor_slices((df.values, target.values))
for feat, targ in dataset.take(3):
  print ('Features: {}, Target: {}'.format(feat, targ))

Features: [1.         0.72868217 0.26829268 0.72862454 0.26686217 0.40210943
 0.15392205 0.64049024 0.71820098 0.         0.         0.
 0.         1.         0.         0.         0.         0.
 0.         0.        ], Target: 0.0
Features: [0.         0.54263566 0.16463415 0.54275093 0.16422287 0.65524061
 0.31327084 0.92510213 0.86156008 0.         0.         0.
 0.         0.         0.         0.         0.         1.
 0.         0.        ], Target: 1.0
Features: [1.         0.37984496 0.57317073 0.37918216 0.57478006 0.
 0.41391219 0.18610985 0.44764582 0.         0.         0.
 0.         0.         1.         0.         0.         0.
 0.         0.        ], Target: 1.0


In [5]:
#shuffle the dataset
train_dataset = dataset.shuffle(len(df)).batch(1)

In [6]:
from keras.models import Sequential
from keras.layers import Dense

# define the keras model
model = Sequential()
model.add(Dense(10, activation='relu'))
model.add(Dense(10, activation='relu'))
model.add(Dense(1, activation='sigmoid'))

In [7]:
# compile the keras model
model.compile(loss='binary_crossentropy', optimizer='adam', metrics=['accuracy'])

In [8]:
# fit the keras model on the dataset
model.fit(train_dataset, epochs=20)

Epoch 1/20

If you intended to run this layer in float32, you can safely ignore this warning. If in doubt, this warning is likely only an issue if you are porting a TensorFlow 1.X model to TensorFlow 2.

To change all layers to have dtype float64 by default, call `tf.keras.backend.set_floatx('float64')`. To change just this layer, pass dtype='float64' to the layer constructor. If you are the author of this layer, you can disable autocasting by passing autocast=False to the base Layer constructor.

1008/1008 [==============================] - 1s 1ms/step - loss: 0.6711 - accuracy: 0.5823
Epoch 2/20
1008/1008 [==============================] - 1s 1ms/step - loss: 0.6420 - accuracy: 0.6349
Epoch 3/20
1008/1008 [==============================] - 1s 1ms/step - loss: 0.6341 - accuracy: 0.6468
Epoch 4/20
1008/1008 [==============================] - 1s 1ms/step - loss: 0.6335 - accuracy: 0.6478
Epoch 5/20
1008/1008 [==============================] - 1s 1ms/step - loss: 0.6331 - accuracy: 0.649